In [1]:
import pandas as pd
import numpy as np
from pathlib import Path
import logging
from new_strategy import Asset, BetSizingMethod, get_bet_sizing
import nbimporter
from backtest import Backtest
from meta_strategy import MetaLabelingStrategy
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.calibration import CalibratedClassifierCV
%load_ext autoreload
%autoreload 2

In [2]:
logging.basicConfig(level=logging.INFO, format="%(asctime)s - %(levelname)s - %(message)s")

# ---------------------- MetaModelHandler ---------------------- #
class MetaModelHandler:
    def __init__(self):
        self.long_model = None
        self.short_model = None
        self.long_scaler = None
        self.short_scaler = None
        self.feature_cols = []

    def train(self, trades_df: pd.DataFrame, feature_cols: list):
        self.feature_cols = feature_cols
        trades_df = trades_df.dropna(subset=feature_cols + ['meta_label'])

        long_trades = trades_df[trades_df['direction'] == 'long']
        short_trades = trades_df[trades_df['direction'] == 'short']

        def preprocess(df):
            X = df[feature_cols]
            y = df['meta_label']
            scaler = StandardScaler()
            X_scaled = scaler.fit_transform(X)
            return X_scaled, y, scaler

        X_long, y_long, self.long_scaler = preprocess(long_trades)
        X_short, y_short, self.short_scaler = preprocess(short_trades)

        self.long_model = CalibratedClassifierCV(LogisticRegression(), method='sigmoid').fit(X_long, y_long)
        self.short_model = CalibratedClassifierCV(LogisticRegression(), method='sigmoid').fit(X_short, y_short)

    def is_trade_approved(self, features: dict, direction: str, threshold: float = 0.5) -> bool:
    # Clean NaNs or infinities in input features
        cleaned = {}
        for k in self.feature_cols:
            val = features.get(k, 0)
            if pd.isna(val) or val in [np.inf, -np.inf]:
                cleaned[k] = 0  # replace invalid values with 0 (or another neutral default)
            else:
                cleaned[k] = val

        df = pd.DataFrame([cleaned])[self.feature_cols]

        if direction == 'long':
            X = self.long_scaler.transform(df)
            prob = self.long_model.predict_proba(X)[0, 1]
        else:
            X = self.short_scaler.transform(df)
            prob = self.short_model.predict_proba(X)[0, 1]

        
        print(f"[MetaModel] Direction: {direction}, Prob: {prob:.3f}, Threshold: {threshold}, Approved: {prob >= threshold}")

        return prob >= threshold


# ---------------------- Helper Functions ---------------------- #
def train_test_split_time_series(trades_df: pd.DataFrame, test_size: float = 0.3):
    trades_df = trades_df.sort_values('entry_time')
    split_idx = int(len(trades_df) * (1 - test_size))
    return trades_df.iloc[:split_idx], trades_df.iloc[split_idx:]

def load_meta_labeled_data(asset: Asset, method: BetSizingMethod, test_size=0.3):
    meta_path = Path(f"data/metalabels/meta_labels_{asset.value}_{method.value}.csv")
    if not meta_path.exists():
        raise FileNotFoundError(f"Meta-labeled file not found: {meta_path}")

    df = pd.read_csv(meta_path, parse_dates=["entry_time", "exit_time"])
    df = df.dropna(subset=["meta_label"])
    df = df.sort_values("entry_time")
    split_idx = int(len(df) * (1 - test_size))
    return df.iloc[:split_idx], df.iloc[split_idx:]

def train_meta_model(train_df: pd.DataFrame, feature_cols: list) -> MetaModelHandler:
    meta_model = MetaModelHandler()
    meta_model.train(train_df, feature_cols)
    return meta_model

# ---------------------- Run Backtest ---------------------- #
def run_metalabel_backtest(asset: Asset, method: BetSizingMethod, feature_cols: list):
    # Load raw price data
    price_data_path = Path(f"data/processed/{asset.value}/combined_data.csv")
    price_data = pd.read_csv(price_data_path, index_col="timestamp", parse_dates=True)

    # Load and split meta-labeled data
    train_df, test_df = load_meta_labeled_data(asset, method)
    print("📊 Diagnostics on training set")
    print("train_df columns:", train_df.columns.tolist())
    print("Unique directions in train_df:", train_df['direction'].unique() if 'direction' in train_df else "Missing 'direction' column")
    print("Rows before dropna:", len(train_df))
    missing_summary = train_df[feature_cols + ['meta_label']].isnull().sum()
    print("Missing values per feature:\n", missing_summary)

    test_start = test_df['entry_time'].min()

    # Train meta model on training data
    meta_handler = train_meta_model(train_df, feature_cols)

    # Restrict price data to test period (to simulate live trading post-training)
    price_data = price_data[price_data.index >= test_start]

    # Get bet sizing method
    past_returns = price_data['close'].pct_change().dropna()
    bet_sizing = get_bet_sizing(method, past_returns)

    # Run strategy with meta-model
    strategy = MetaLabelingStrategy(price_data, asset.value, bet_sizing, method, meta_model_handler=meta_handler,feature_cols=feature_cols)
    strategy.generate_signals()
    strategy.simulate_trades()

    # Backtest results
    backtest = Backtest(strategy)
    backtest.run_analysis()
    backtest.print_summary()
    print(f"\n🧮 Total Trades Rejected by Meta-Model: {strategy.rejected_trades}")

    # Save filtered trades to CSV for inspection
    output_dir = Path("data/results_metalabel")
    output_dir.mkdir(parents=True, exist_ok=True)

    filename = f"trades_meta_filtered_{asset.value}_{method.value}.csv"
    filtered_trades_df = strategy.get_trade_data()
    filtered_trades_df.to_csv(output_dir / filename, index=False)

    logging.info(f"Saved filtered trades to: {output_dir / filename}")

# --- Print key performance metrics ---
    metrics = backtest.results['sessions']
    print(f"\n--- Meta-Filtered Performance Summary ({asset.value}, {method.value}) ---")
    for session, result in metrics.items():
        m = result['metrics']
        print(f"\n[{session.upper()} SESSION]")
        print(f"Total PnL: ${m.total_pnl:,.2f}")
        print(f"Return: {m.total_return_pct:.2f}%")
        print(f"Sharpe Ratio: {m.sharpe:.2f}" if m.sharpe else "Sharpe: n/a")
        print(f"Win Rate: {m.win_rate:.2%}")
        print(f"Max Drawdown: {m.max_drawdown_pct:.2f}%")


# ---------------------- Main Entry ---------------------- #
if __name__ == "__main__":
    asset = Asset.XAUUSD
    method = BetSizingMethod.FIXED
    feature_cols = ["atr_14", "attempt", "duration_minutes", "ref_close"]

    run_metalabel_backtest(asset, method, feature_cols)


📊 Diagnostics on training set
train_df columns: ['asset', 'entry_time', 'exit_time', 'entry_price', 'exit_price', 'direction', 'session', 'attempt', 'status', 'position_size', 'risk_amount', 'pnl', 'return_pct', 'holding_time', 'ref_close', 'date', 'day_of_week_x', 'equity_at_entry', 'duration_minutes', 'atr_value', 'capital_curve', 'meta_label', 'rolling_pnl_5', 'rolling_winrate_5', 'rolling_volatility_5', 'atr_14', 'ma_14', 'ma_30', 'ma_100', 'day_of_week_y', 'week_number', 'max_price_14', 'min_price_14', 'max_price_30', 'min_price_30', 'max_price_100', 'min_price_100']
Unique directions in train_df: ['long' 'short']
Rows before dropna: 3654
Missing values per feature:
 atr_14              73
attempt              0
duration_minutes     0
ref_close            0
meta_label           0
dtype: int64


2025-05-22 18:24:21,812 - INFO - Strategy initialized for XAUUSD using FixedFractionalBetSizing


[MetaModel] Direction: short, Prob: 0.504, Threshold: 0.15, Approved: True
[MetaModel] Direction: short, Prob: 0.492, Threshold: 0.15, Approved: True
[MetaModel] Direction: long, Prob: 0.528, Threshold: 0.15, Approved: True
[MetaModel] Direction: long, Prob: 0.519, Threshold: 0.15, Approved: True
[MetaModel] Direction: long, Prob: 0.511, Threshold: 0.15, Approved: True
[MetaModel] Direction: short, Prob: 0.504, Threshold: 0.15, Approved: True
[MetaModel] Direction: short, Prob: 0.504, Threshold: 0.15, Approved: True
[MetaModel] Direction: short, Prob: 0.492, Threshold: 0.15, Approved: True
[MetaModel] Direction: short, Prob: 0.504, Threshold: 0.15, Approved: True
[MetaModel] Direction: short, Prob: 0.504, Threshold: 0.15, Approved: True
[MetaModel] Direction: short, Prob: 0.492, Threshold: 0.15, Approved: True
[MetaModel] Direction: long, Prob: 0.526, Threshold: 0.15, Approved: True
[MetaModel] Direction: long, Prob: 0.526, Threshold: 0.15, Approved: True
[MetaModel] Direction: short, 

2025-05-22 18:25:29,587 - INFO - Exported detailed trades to data/results/trades_detailed_XAUUSD_fixed.csv
2025-05-22 18:25:29,648 - INFO - Saved backtest results to data/results/backtest_results_XAUUSD_fixed.txt
2025-05-22 18:25:29,778 - INFO - Saved filtered trades to: data/results_metalabel/trades_meta_filtered_XAUUSD_fixed.csv



=== XAUUSD Backtest Results ===
Bet Sizing Strategy: fixedfractionalbetsizing

Period: 2023-05-11 to 2024-11-22

ASIAN Session Performance:
Initial Capital: $100,000.00
Final Capital: $100,173.48
Total PnL: $173.48
Return: 0.17%
Win Rate: 48.68% (222W/234L)
Max Drawdown: 1.18%
Sharpe Ratio: 0.05
Skewness: 0.034
Excess Kurtosis: -0.884

Attempt Analysis:

  Attempt 1:
  Trades: 401 (87.9% of session trades)
  PnL: $585.00
  Win Rate: 48.88% (196W/205L)
  Average Win: $54.99
  Average Loss: $-50.46

  Attempt 2:
  Trades: 50 (11.0% of session trades)
  PnL: $-67.91
  Win Rate: 52.00% (26W/24L)
  Average Win: $32.13
  Average Loss: $-37.63

  Attempt 3:
  Trades: 5 (1.1% of session trades)
  PnL: $-343.61
  Win Rate: 0.00% (0W/5L)
  Average Win: $0.00
  Average Loss: $-68.72

LONDON Session Performance:
Initial Capital: $100,000.00
Final Capital: $103,774.76
Total PnL: $3,774.76
Return: 3.77%
Win Rate: 49.73% (277W/280L)
Max Drawdown: 1.69%
Sharpe Ratio: 1.25
Skewness: 0.101
Excess Kurto